In [ ]:
"""
간단한 텍스트 기반 객체 탐지 예제
==================================
이 노트북은 OWLv2 모델을 사용한 기본적인 텍스트 기반 객체 탐지 예제입니다.
텍스트 프롬프트를 사용하여 이미지에서 객체를 탐지하는 방법을 보여줍니다.

"""

import json
from PIL import Image, ImageDraw, ImageFont
import torch
from torchvision.ops import nms
from transformers import Owlv2Processor, Owlv2ForObjectDetection

# ==================== 시스템 초기화 ====================
# 0. 디바이스 설정 (GPU 우선, 없으면 CPU 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")  # CUDA 사용 가능하면 'cuda' 출력

# 1. OWLv2 모델 & 프로세서 로드
# Google의 Open-Vocabulary Object Detection 모델
processor = Owlv2Processor.from_pretrained(
    "google/owlv2-large-patch14-ensemble",
    image_size=1008,    # OWLv2 최적 입력 크기 (14×72 패치)
)
model = Owlv2ForObjectDetection.from_pretrained(
    "google/owlv2-large-patch14-ensemble"
).to(device).eval()  # 평가 모드로 설정

print("OWLv2 모델 로딩 완료!")



/home/dromii4/dromii/owlv2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [ ]:
# ==================== 설정 및 경로 ====================
import json
from pathlib import Path
from torchvision.ops import nms
from PIL import Image, ImageDraw, ImageFont

# 디렉토리 설정
IMAGE_DIR    = Path("encumbrance_labels/encumbrance")        # 원본 이미지 폴더
OUTPUT_DIR   = Path("encumbrance_labels/simple_prediction")  # 결과 저장 폴더

# ==================== 탐지할 객체 클래스 ====================
# 6가지 주요 객체 클래스 정의
# 건축물, 컨테이너, 비닐하우스, 논/밭, 수목, 분묘
TEXT_QUERIES = [
    "Tomb",        # 무덤/고분
    "Tree",        # 나무
    "Greenhouse",  # 온실/비닐하우스
    "Building",    # 건물
    "Field",       # 밭/논
    "Container"    # 컨테이너
]

print(f"탐지 대상 클래스: {TEXT_QUERIES}")
print(f"입력 디렉토리: {IMAGE_DIR}")
print(f"출력 디렉토리: {OUTPUT_DIR}")

# ==================== 시각화 및 하이퍼파라미터 ====================
# 클래스별 시각화 색상 (RGB 값)
COLOR_MAP = {
    "Tomb":       (0,   0, 255),  # 파란색 - 무덤
    "Tree":       (0, 255,   0),  # 초록색 - 나무
    "Greenhouse": (255, 0,   0),  # 빨간색 - 온실
    "Building":   (0, 255, 255),  # 청록색 - 건물
    "Field":      (255, 0, 255),  # 자홍색 - 밭
    "Container":  (255, 255,   0), # 노란색 - 컨테이너
}

# 탐지 임계값 설정
THRESHOLD     = 0.2  # 신뢰도 임계값 (0.2 이상만 탐지)
IOU_THRESHOLD = 0.3  # NMS IoU 임계값 (0.3 이상 겹치는 박스 제거)
TARGET_SIZE   = 1008  # OWLv2 최적 입력 크기로 수정

# ==================== 초기 설정 ====================
# 출력 폴더 생성
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
font = ImageFont.load_default()  # 텍스트 표시용 폰트

# ==================== 메인 처리 루프 ====================
# 이미지 디렉토리의 모든 이미지 파일 처리
for img_path in IMAGE_DIR.glob("*"):
    # 지원하는 이미지 형식만 처리 (.jpg, .jpeg, .png)
    if img_path.suffix.lower() not in {".jpg", ".jpeg", ".png"}:
        continue

    print(f"처리 중: {img_path.name}")

    # ==================== 1단계: 이미지 전처리 ====================
    # 원본 이미지 로드 및 RGB 변환
    image = Image.open(img_path).convert("RGB")
    # OWLv2 모델 입력 크기로 리사이징 (1008x1008)
    image = image.resize((TARGET_SIZE, TARGET_SIZE), Image.BILINEAR)
    w, h = image.size  # w == h == 1008

    # ==================== 2단계: OWLv2 모델 입력 준비 ====================
    # 텍스트 쿼리와 이미지를 프로세서에 전달하여 모델 입력 생성
    inputs = processor(text=[TEXT_QUERIES], images=image, return_tensors="pt")
    # GPU로 데이터 이동 (가능한 경우)
    for k, v in inputs.items():
        if hasattr(v, "to"):
            inputs[k] = v.to(device)

    # ==================== 3단계: 모델 추론 및 후처리 ====================
    # GPU 메모리 사용량 최적화를 위한 no_grad 컨텍스트
    with torch.no_grad():
        outputs = model(**inputs)  # OWLv2 모델 추론
        
    # 모델 출력을 바운딩 박스, 점수, 라벨로 변환
    results = processor.post_process_object_detection(
        outputs=outputs,
        target_sizes=torch.tensor([[h, w]], device=device),  # 원본 이미지 크기
        threshold=THRESHOLD  # 신뢰도 임계값 적용
    )[0]

    # 결과 분리
    boxes, scores, labels = (
        results["boxes"],    # 바운딩 박스 좌표 [x1, y1, x2, y2]
        results["scores"],   # 신뢰도 점수
        results["labels"],   # 클래스 라벨 인덱스
    )
    
    # ==================== 4단계: Non-Maximum Suppression (NMS) ====================
    # 겹치는 박스들 중에서 가장 높은 점수의 박스만 유지
    keep = nms(boxes, scores, IOU_THRESHOLD)
    boxes, scores, labels = boxes[keep], scores[keep], labels[keep]

    # ==================== 5단계: 시각화 및 결과 저장 ====================
    draw = ImageDraw.Draw(image)  # 이미지에 그리기 객체 생성
    records = []  # JSON 저장용 결과 리스트

    # 각 탐지된 객체에 대해 시각화 및 기록
    for box, score, li in zip(boxes, scores, labels):
        label = TEXT_QUERIES[li]  # 클래스 인덱스를 이름으로 변환
        xmin, ymin, xmax, ymax = box.tolist()
        
        # ==================== 박스 좌표 클램핑 ====================
        # 이미지 경계를 벗어나는 박스 좌표를 이미지 내부로 제한
        xmin, ymin = max(0, xmin), max(0, ymin)
        xmax, ymax = min(w, xmax), min(h, ymax)
        
        # 클래스별 색상 가져오기
        color = COLOR_MAP.get(label, (255,255,255))

        # ==================== 바운딩 박스 그리기 ====================
        draw.rectangle([xmin, ymin, xmax, ymax], outline=color, width=3)
        
        # ==================== 라벨 텍스트 표시 ====================
        txt = f"{label}: {score:.2f}"  # 클래스명과 신뢰도 점수
        # 텍스트 배경 박스 계산
        x0, y0, x1, y1 = draw.textbbox((xmin, ymin), txt, font=font)
        # 텍스트 배경 그리기
        draw.rectangle([x0, y0, x1, y1], fill=color)
        # 흰색 텍스트 그리기
        draw.text((xmin, ymin), txt, fill=(255,255,255), font=font)

        # ==================== JSON 기록용 데이터 추가 ====================
        records.append({
            "label": label,
            "score": float(score),
            "box": [round(xmin,2), round(ymin,2), round(xmax,2), round(ymax,2)]
        })

    # ==================== 6단계: 결과 파일 저장 ====================
    out_img  = OUTPUT_DIR / img_path.name  # 시각화된 이미지 저장 경로
    out_json = OUTPUT_DIR / f"{img_path.stem}.json"  # JSON 결과 저장 경로
    
    # 시각화된 이미지 저장
    image.save(out_img)
    # JSON 결과 저장 (UTF-8 인코딩, 한글 지원)
    with open(out_json, "w", encoding="utf-8") as fp:
        json.dump(records, fp, ensure_ascii=False, indent=2)

    print(f"[OK] {img_path.name} → {out_img.name}, {out_json.name}")

[OK] DJI_0951.JPG → DJI_0951.JPG, DJI_0951.json
[OK] DJI_0691.JPG → DJI_0691.JPG, DJI_0691.json
[OK] DJI_0033.JPG → DJI_0033.JPG, DJI_0033.json
[OK] DJI_0559.jpg → DJI_0559.jpg, DJI_0559.json
[OK] DJI_0934.jpg → DJI_0934.jpg, DJI_0934.json
[OK] DJI_0923.jpg → DJI_0923.jpg, DJI_0923.json
[OK] DJI_0008.JPG → DJI_0008.JPG, DJI_0008.json
[OK] DJI_0933.jpg → DJI_0933.jpg, DJI_0933.json
[OK] DJI_0866.JPG → DJI_0866.JPG, DJI_0866.json
[OK] DJI_0036.JPG → DJI_0036.JPG, DJI_0036.json
[OK] DJI_0928.jpg → DJI_0928.jpg, DJI_0928.json
